In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import re
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import numpy as np
import os

In [ ]:
path = '/gpfs/data/ssa/users/d602145/Workspace/scratch/Porosity/ETH/'
os.chdir(path)

In [ ]:
from Lib.Data import PorosityDistribution, extract_microstructures, extract_porosities_points
from Lib.Datasets import  PorosityDataset

In [ ]:
sample_path = os.getcwd()+'/Job_Assignment_Data/Job_Assignment_Data/'

In [ ]:
extracted_porosities, density_set = extract_porosities_points(sample_path, keep_density_doubles=False)

In [ ]:
df = extracted_porosities.groupby(by='density').count()

In [ ]:
df.head()

In [ ]:
px.scatter(df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
df['density'] = df.index

In [ ]:
X = df['density'].values.reshape(-1,1)
y = df['sample_number'].values.reshape(-1,1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
st_scale = StandardScaler()
y_train_sc = st_scale.fit_transform(y_train)
y_test_sc = st_scale.transform(y_test)

In [ ]:
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [ ]:
X_test_poly.shape

In [ ]:
model = LinearRegression()

model.fit(X_train_poly,y_train_sc)

y_pred_test = model.predict(X_test_poly)
y_pred_train = model.predict(X_train_poly)

In [ ]:
y_pred_test = st_scale.inverse_transform(y_pred_test)
y_test = st_scale.inverse_transform(y_test_sc)

y_pred_test = y_pred_test.reshape(y_pred_test.shape[0])
y_test = y_test.reshape(y_test.shape[0])

y_pred_train = st_scale.inverse_transform(y_pred_train)
y_train = st_scale.inverse_transform(y_train_sc)

y_pred_train = y_pred_train.reshape(y_pred_train.shape[0])
y_train = y_train.reshape(y_train.shape[0])

X_train = X_train.reshape(X_train.shape[0])
X_test = X_test.reshape(X_test.shape[0])

In [ ]:
fig = px.scatter(x=y_pred_train, y=y_train,
                 opacity=0.5,  # Set opacity for better visualization
                 trendline="ols",  # Add trendline using Ordinary Least Squares
                 title="Regression Plot: Actual vs. Predicted Density")

# Add ideal line (y = x)
x_range = np.linspace(min(y_test_sc), max(y_test_sc), 100)
fig.add_scatter(x=x_range, y=x_range, mode='lines',
                line=dict(color='red', dash='dash'),
                name='Ideal Line (y = x)')

fig.update_layout(
    xaxis_title="Actual Density",
    yaxis_title="Predicted Density"
)

fig.show()


In [ ]:
fig = px.scatter(x=y_pred_test, y=y_test,
                 opacity=0.5,  # Set opacity for better visualization
                 trendline="ols",  # Add trendline using Ordinary Least Squares
                 title="Regression Plot: Actual vs. Predicted Density")

# Add ideal line (y = x)
x_range = np.linspace(min(y_test_sc), max(y_test_sc), 100)
fig.add_scatter(x=x_range, y=x_range, mode='lines',
                line=dict(color='red', dash='dash'),
                name='Ideal Line (y = x)')

fig.update_layout(
    xaxis_title="Actual Density",
    yaxis_title="Predicted Density"
)

fig.show()

In [ ]:

fig = px.scatter(x=X_train, y=y_train,
                 opacity=0.5,  # Set opacity for better visualization
                 title="Regression Plot: Actual vs. Predicted Density")

fig.add_scatter(x=X_test, y=y_test,
                 opacity=0.5, mode='markers')  # Set opacity for better visualization)

fig.show()